In [ ]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install evaluate --quiet
!pip install sentencepiece --quiet
!pip install accelerate --quiet
!pip install rouge_score --quiet
!pip install bert_score --quiet
#!pip install transformers[deepspeed] --quiet
!pip install deepspeed --quiet

     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 71.5 MB/s 
     |████████████████████████████████| 163 kB 79.2 MB/s 
     |████████████████████████████████| 441 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 83.9 MB/s 
     |████████████████████████████████| 212 kB 77.7 MB/s 
     |████████████████████████████████| 127 kB 87.3 MB/s 
     |████████████████████████████████| 115 kB 82.7 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 148 kB 5.0 MB/s 
     |████████████████████████████████| 60 kB 3.6 MB/s 
     |████████████████████████████████| 665 kB 5.2 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 120 kB 74.3 MB/s 


In [ ]:
#ds_report
from deepspeed.runtime.zero.stage_1_and_2 import estimate_zero2_model_states_mem_needs_all_live
from transformers import MT5ForConditionalGeneration, T5Tokenizer
estimate_zero2_model_states_mem_needs_all_live(model = MT5ForConditionalGeneration.from_pretrained('google/mt5-base'), 
                                                                                    num_gpus_per_node=1, num_nodes=1, additional_buffer_factor=1.5)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 1 GPU per node.
SW: Model with 582M total params.
  per CPU  |  per GPU |   Options
   13.02GB |   1.08GB | offload_optimizer=cpu 
    3.25GB |  10.85GB | offload_optimizer=none


In [ ]:
from deepspeed.runtime.zero.stage3 import estimate_zero3_model_states_mem_needs_all_live
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-base')
estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=1, num_nodes=1, additional_buffer_factor=1.5)

Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 1 GPU per node.
SW: Model with 582M total params, 192M largest layer params.
  per CPU  |  per GPU |   Options
   14.64GB |   0.72GB | offload_param=cpu , offload_optimizer=cpu , zero_init=1
   14.64GB |   0.72GB | offload_param=cpu , offload_optimizer=cpu , zero_init=0
   13.02GB |   1.80GB | offload_param=none, offload_optimizer=cpu , zero_init=1
   13.02GB |   1.80GB | offload_param=none, offload_optimizer=cpu , zero_init=0
    1.07GB |  10.48GB | offload_param=none, offload_optimizer=none, zero_init=1
    3.25GB |  10.48GB | offload_param=none, offload_optimizer=none, zero_init=0


In [ ]:
#!pip install torch==1.10.1+cu11.0 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

#!pip3 install torch==1.10.1+cu113 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
!cp /content/drive/MyDrive/MscThesis/Evaluation_code/Bartscore.py /content

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!cat /proc/meminfo

MemTotal:       13297228 kB
MemFree:         9152568 kB
MemAvailable:   11974108 kB
Buffers:           99244 kB
Cached:          2833732 kB
SwapCached:            0 kB
Active:           762440 kB
Inactive:        3110316 kB
Active(anon):        980 kB
Inactive(anon):   866624 kB
Active(file):     761460 kB
Inactive(file):  2243692 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             10788 kB
Writeback:             0 kB
AnonPages:        939916 kB
Mapped:           634700 kB
Shmem:              1296 kB
KReclaimable:     115680 kB
Slab:             152672 kB
SReclaimable:     115680 kB
SUnreclaim:        36992 kB
KernelStack:        6064 kB
PageTables:        13764 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648612 kB
Committed_AS:    4027272 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       50656 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
import os

#import os.path
import pickle
import pandas as pd
import numpy as np
import re
import torch 
import nltk

from transformers import AutoTokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline, AutoModelForSeq2SeqLM
from transformers import MT5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq
import datasets
import evaluate
import accelerate
import deepspeed

#import Evaluation_Code.Parent as parent ## code for PARENT metric
import Bartscore as bartscore ## code for Bartscore
import gc
import json
from ast import literal_eval

from google.colab import files


In [ ]:
############################################################################## Begin Environment Setup ######################################################################################

# DeepSpeed requires a distributed environment even when only one process is used.
# This emulates a launcher in the notebook

os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

def ensure_cuda_compatability():
    print(f'Torch version: {torch.__version__}')
    print(f'Cuda version: {torch.version.cuda}')
    print(f'Cudnn version: {torch.backends.cudnn.version()}')
    print(f'Is cuda available: {torch.cuda.is_available()}')
    print(f'Number of cuda devices: {torch.cuda.device_count()}')
    print(f'Current default device: {torch.cuda.current_device()}')
    print(f'First cuda device: {torch.cuda.device(0)}')
    print(f'Name of the first cuda device: {torch.cuda.get_device_name(0)}\n\n')
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    #Ensure we are really working with full capacity
    gc.collect() 
    torch.cuda.empty_cache()

############################################################################## End Environment Setup ######################################################################################

############################################################################## Begin Model and Dataset Setup ######################################################################################

def preprocess_model(model_name):
    """
    Setup the model and tokenizer for preprocessing. This will be a pre-trained model collected from huggingface
    """

    #with deepspeed.zero.Init():
      #config = T5Config.from_pretrained("t5-small")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    #model.cuda()

    print('LOGGING: preprocess_model DONE \n')
    return model, tokenizer


def load_CACAPO_data():
    """
    This function retrieves the csv files and creates a dataset
    """
    print('LOGGING: load_CACAPO_data DONE \n')

    return datasets.load_dataset("/content/drive/MyDrive/MscThesis/Data/Cleaned_data", data_files={"train": "Train.csv", "dev": "Dev.csv", "test": "Test.csv"})


def preprocess_data(data):
    """
    Tokenize the data
    """
    max_length = 256
    RDFs = data["input"]
    texts = data["output"]

    ## When converting a pandas df to csv (used for loading dataset), a list of lists can transform to a long string
    ## Here we convert it back with literal_eval

    for rdf_iteration, rdf in enumerate(RDFs):
        RDFs[rdf_iteration] = literal_eval(rdf)

    model_inputs = tokenizer(RDFs, truncation=True, padding='max_length', return_tensors='pt',  max_length=max_length, is_split_into_words=True)
    
    # specially for seq2seq tokenizer, "Temporarily sets the tokenizer for encoding the targets. Useful for tokenizer associated to sequence-to-sequence models that need a slightly different processing for the labels."
    with tokenizer.as_target_tokenizer():
        target_texts = tokenizer(texts, padding='max_length', truncation=True, return_tensors='pt',  max_length=max_length).input_ids
        
    model_inputs["labels"] = target_texts

    return model_inputs



def transform_datasets(dataset):
    """
    After loading in and creating the initial dataset, the text data is transformed, by tokenizing the input and output texts. The initial dataset is also split into train,val,test for training use.
    NOTE That the test set will not be preprocessed here yet, this will be done in a different function
    """

    ## Create smaller versions of the dataset
    # small_train = dataset["train"].shard(num_shards = 256, index = 0)
    # small_val = dataset["dev"].shard(num_shards = 256, index = 0)
    # small_test = dataset["test"].shard(num_shards = 256, index = 0)

    small_train = dataset["train"]
    small_val = dataset["dev"]
    small_test = dataset["test"]

    # to use the actual articles for evaluation
    true_articles_test = small_test['output']
    # The Parent Metric requires the original RDFs
    test_rdf_input = small_test['input']


    ## Process the data in batches
    small_train = small_train.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
    small_val = small_val.map(preprocess_data, batched=True, remove_columns=dataset["dev"].column_names)
    small_test = small_test.map(preprocess_data, batched=True, remove_columns=dataset["test"].column_names)

    # transform the datasets into torch sensors, as the model will expect this format
    small_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels']) #, device="cuda")
    small_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels']) #, device="cuda")
    small_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels']) #, device="cuda")

    print('LOGGING: transform_datasets DONE \n')

    return small_train, small_val, small_test, true_articles_test, test_rdf_input

############################################################################## End Model and Dataset Setup ######################################################################################

############################################################################## Begin Evaluation Setup######################################################################################


def load_eval_metrics():
    """
    Loads in all metrics that will be used later on during evaluation. This is seperated to not load in the metrics a dozen of times during training.
    """
    bleu = datasets.load_metric("bleu")
    rouge = evaluate.load('rouge')
    meteor = evaluate.load('meteor')
    perplexity = evaluate.load("perplexity", module_type="metric")
    bertscore = evaluate.load("bertscore")
    bart_scorer = bartscore.BARTScorer(device='cuda:0',checkpoint='facebook/bart-base')

    print('LOGGING: load_eval_metrics DONE \n')

    return bleu, rouge, meteor, perplexity, bertscore, bart_scorer


def postprocess_text(preds, labels):
    """
    Supplementary Method called in decode_text.

    Returns list of split decoded labels and predictions for evaluation
    """
    preds = [pred.split() for pred in preds]
    labels = [[label.split()] for label in labels]
    return preds, labels



def decode_text(predictions, labels):
    """
    Supplementary Method called in compute_metrics.

    Returns decoded labels and predictions for evaluation
    """
    if isinstance(predictions, tuple):
            predictions = predictions[0]

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return decoded_preds, decoded_labels

############################################################################## End Evaluation Setup######################################################################################

############################################################################## Begin Evaluation######################################################################################

def evaluate_texts(decoded_preds, decoded_labels):
    """
    Calculates metrics given a list of decoded predictions and decoded labels
    """
    #post_process for BLEU
    blue_preds, blue_labels = postprocess_text(decoded_preds,  decoded_labels)

    # setup metrics for use
    bleu, rouge, meteor, perplexity, bertscore, bart_scorer = load_eval_metrics()

    # Calculate the metrics
    print(f'\n LOGGING: Calculating Blue')
    bleu_output = bleu.compute(predictions=blue_preds, references=blue_labels)
    print(f'\n LOGGING: Calculating Rouge')
    rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'\n LOGGING: Calculating Meteor')
    meteor_output = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'\n LOGGING: Calculating Perplexity')
    perp_output = perplexity.compute(predictions=decoded_preds, model_id='gpt2')
    print(f'\n LOGGING: Calculating Bertscore')
    bertscore_output = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    print(f'\n LOGGING: Calculating Bartscore')
    bart_scores_output = bart_scorer.score(srcs=decoded_preds, tgts=decoded_labels, batch_size=16)

    return bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output



def compute_metrics(pred):
    """"
    Metrics to be evaluated during training and validation
    Metrics used: BLEU, ROUGE, METEOR, Bertscore, BARTScore
    """
    # decode the predictions and labels for eval
    predictions, labels = pred
    decoded_preds, decoded_labels = decode_text(predictions, labels)

    bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output = evaluate_texts(decoded_preds, decoded_labels)

    ## Huggingsface trainer requires a dict if multiple metrics are used
    return {"blue_output": bleu_output, "rouge_output": rouge_output, "meteor_results": meteor_output,
             "perp_output": perp_output, "bertscore_output": bertscore_output,  "bart_scores_output": bart_scores_output}


############################################################################## End Evaluation Section######################################################################################

############################################################################## Begin Huggingface Trainer Setup ######################################################################################

def set_training_args(model_name, learning_rate, num_train_epochs, evaluation_strategy, generation_num_beams, generation_max_length,
                         gradient_accumulation_steps, per_device_train_batch_size, per_device_eval_batch_size):
    """
    Setup the training arguments that will be used during training.
    """
    model_dir = f"/content/drive/MyDrive/MscThesis/Results/{model_name}"

    training_args = Seq2SeqTrainingArguments(
                output_dir=model_dir,
                learning_rate=learning_rate,
                do_eval=True, # will be set to true if evaluation strategy is set
                do_predict=True, #Whether to run predictions on the test set or not.
                num_train_epochs=num_train_epochs,
                evaluation_strategy= evaluation_strategy,
                eval_steps= 2500, # Number of update steps between two evaluations if evaluation_strategy="steps". Will default to the same value as logging_steps if not set.
                save_steps=500, # Number of updates steps before two checkpoint saves if save_strategy="steps".
                #max_steps=10, # the total number of training steps to perform
                save_total_limit= 10, # the maximum number of models to keep before deleting the oldest one
                predict_with_generate=True, # Whether to use generate to calculate generative metrics (ROUGE, BLEU).
                generation_num_beams=generation_num_beams,  #The num_beams to use on each evaluation loop when predict_with_generate=True. Will default to the num_beams value of the model configuration
                gradient_checkpointing=True, #
                generation_max_length=generation_max_length,
                gradient_accumulation_steps=gradient_accumulation_steps, #Number of updates steps to accumulate the gradients for, before performing a backward/update pass
                per_device_train_batch_size=per_device_train_batch_size, #The batch size per GPU/TPU core/CPU for training.
                per_device_eval_batch_size=per_device_eval_batch_size, #The batch size per GPU/TPU core/CPU for evaluation.
                #optim="adafactor", #The optimizer to use: adamw_hf, adamw_torch, adamw_apex_fused, or adafactor.
                #report_to="tensorboard",
                #fp16=True,
                bf16=True, ## should now be possible with rtx 3070
                tf32=True, #--> moet dan ook torch.backends.cuda.matmul.allow_tf32 = True
                auto_find_batch_size=True, #Whether to find a batch size that will fit into memory automatically through exponential decay,
                                            # avoiding CUDA Out-of-Memory errors. Requires accelerate to be installed (pip install accelerate)
                #deepspeed = "/content/drive/MyDrive/MscThesis/Deepseed_zero_3.json",
                deepspeed = "/content/drive/MyDrive/MscThesis/Deepspeed_zero_2_test.json",
                #deepspeed = "/content/drive/MyDrive/MscThesis/Dspeed_zero_test.json",

                eval_accumulation_steps=5  #Number of predictions steps to accumulate the output tensors for, 
                                            # before moving the results to the CPU. If left unset, the whole predictions are accumulated on GPU/TPU 
                                            # before being moved to the CPU (faster but requires more memory).
       )

    print('LOGGING: set_training_args DONE \n')

    return training_args


def get_clean_model(model_name):
    """
    Simple function to ensure that a new model is used for finetuning
    """
    # with deepspeed.zero.Init():
    #   model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()
    return AutoModelForSeq2SeqLM.from_pretrained(model_name)#.cuda()

def set_trainer(model_name, training_args, train_ds, val_ds, tokenizer):
    """
    Initializes a trainer
    Takes in: Model name, training arguments, training dataset, validation dataset, and tokenizer
    Returns: Trainer instance
    """
    clean_model = get_clean_model(model_name)
    data_collator = DataCollatorForSeq2Seq(tokenizer)
    trainer = Seq2SeqTrainer(
                model=clean_model,
                args=training_args,
                train_dataset=train_ds,
                eval_dataset=val_ds,
                compute_metrics=compute_metrics,
                data_collator = data_collator,
                tokenizer=tokenizer
                )

    print('LOGGING: set_trainer DONE \n')

    return trainer
############################################################################## End Huggingface Trainer Setup ######################################################################################

############################################################################## Begin Train and Save ######################################################################################


def train_and_save(trainer, path_model_name):
    trainer.train()
    trainer.save_model(f"/content/drive/MyDrive/MscThesis/Models/{path_model_name}")

    print('LOGGING: train_and_save DONE \n')

############################################################################## End Train and Save ######################################################################################


############################################################################## Begin Evaluation Process ######################################################################################

def get_saved_model(path_model_name):

    saved_model_path = f"/content/drive/MyDrive/MscThesis/Models/{path_model_name}"
    #print(f'path to folder is: {saved_model_path}')

    saved_model = T5ForConditionalGeneration.from_pretrained(saved_model_path, local_files_only=True)
    saved_model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(saved_model_path ,local_files_only=True)
    return saved_model, tokenizer

def generate_predictions(saved_model, test_set):
    """
    Generates predictions based on the test set, returns a list of predictions and the corresponding "true" articles
    """
    encoded_inputs = test_set.remove_columns("labels")

    # set-up a dataloader to load in the tokenized test dataset
    test_dataloader = torch.utils.data.DataLoader(encoded_inputs,  batch_size=16) #pin_memory=True, --> levert error op: RuntimeError: cannot pin 'torch.cuda.LongTensor' only dense CPU tensors can be pinned

    # generate text for each batch
    all_predictions = []
    for i,batch in enumerate(test_dataloader):
        predictions = saved_model.generate(**batch, max_new_tokens = 100, do_sample=True, num_beams = 5, top_p=0.7, repetition_penalty = 1.3) # .to(device) --> RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)
        all_predictions.append(predictions)

    # flatten predictions
    all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

    print('LOGGING: generate_predictions DONE \n')


    return all_predictions_flattened


def decode_predictions(predictions, tokenizer):
    """
    Decode the predictions made by the model
    """
    decoded_predictions = []

    for iteration, prediction in enumerate(predictions):
        decoded_predictions.append(tokenizer.decode(prediction,skip_special_tokens=True))

    print('LOGGING: decode_predictions DONE \n')

    return decoded_predictions



def evaluate_test_set(path_model_name, test_set, true_articles_test, test_rdf_input):
    """
    Transforms test set, retrieves predictions, and evaluates these predictions
    """
    saved_model, saved_tokenizer = get_saved_model(path_model_name)

    predictions = generate_predictions(saved_model, test_set)

    decoded_test_predictions = decode_predictions(predictions, saved_tokenizer)

    bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output = evaluate_texts(decoded_test_predictions, true_articles_test)

    ## Huggingsface trainer requires a dict if multiple metrics are used
    evaluation_results = {"blue_output": bleu_output, "rouge_output": rouge_output, "meteor_results": meteor_output, "perp_output": perp_output, "bertscore_output": bertscore_output, "bart_scores_output": bart_scores_output}

    log_results(path_model_name, evaluation_results)

    ##Additional PARENT evaluation
    tables = test_rdf_input
    references = true_articles_test
    generations = decoded_test_predictions
    parent_attempt(path_model_name, generations, references, tables)
    
    return evaluation_results


def write_to_text_parent(path_model_name, decoded_predictions, true_articles, rdfs):
    """
    Parent metric requires text files to work
    """

    with open(f'/content/drive/MyDrive/MscThesis/Parent_test/{path_model_name}_true_articles.txt', 'w', encoding='utf-8') as f:
        for articles in true_articles:
            f.write(f'{articles} \n')

    with open(f'/content/drive/MyDrive/MscThesis/Parent_test/{path_model_name}_decode_predictions.txt', 'w', encoding='utf-8') as f:
        for predictions in decoded_predictions:
            f.write(f'{predictions} \n')

    with open(f'/content/drive/MyDrive/MscThesis/Parent_test/{path_model_name}_rdfs.txt', 'w', encoding='utf-8') as f:
        for pairs in rdfs:
            f.write(f'{pairs} \n')


def prepare_inputs_parent(RDFs):
    """
    Cleans the RDF pairs and transforms them in the proper format so that the parent module can calculate with it.
    """

    attribute_value_pairs = []

    for iteration, inputRDF in enumerate(RDFs):
        split_RDF = inputRDF.split(", ")
        entry=[]
        for connected_pair in split_RDF:
            if '[' in connected_pair:
                connected_pair = connected_pair.replace('[', '')
            if ']' in connected_pair:
                connected_pair = connected_pair.replace(']', '')
            if '_' in connected_pair:
                connected_pair = connected_pair.replace('_', ' ')
            split_pair = tuple(connected_pair.split(' | '))
            entry.append((split_pair))
        attribute_value_pairs.append(entry)
    return attribute_value_pairs


def parent_attempt(path_model_name, generations, references, rdfs):
    """
    The Parent metric needs special treatment, as it only accepts specific inputs and file types.
    """
    prepared_rdfs = prepare_inputs_parent(rdfs)
    write_to_text_parent(path_model_name, generations, references, prepared_rdfs)

    !python -i "/content/drive/MyDrive/MscThesis/Evaluation_code/Parent.py" --references f"/content/drive/MyDrive/MscThesis/Parent_test/{path_model_name}_true_articles.txt" \
                                                                              --generations f"/content/drive/MyDrive/MscThesis/Parent_test/{path_model_name}_decode_predictions.txt"  \
                                                                              --tables f"/content/drive/MyDrive/MscThesis/Parent_test/{path_model_name}_rdfs.txt"

def log_results(path_model_name, results):
    with open(f'/content/drive/MyDrive/MscThesis/Logging_Results/{path_model_name}_logResults.json', 'w') as convert_file:
        convert_file.write(json.dumps(results))

############################################################################## End Evaluation Process ######################################################################################

############################################################################## Begin Full fine-tune setup######################################################################################

def fine_tune_model(model_name):
    # ensure cuda compatability
    ensure_cuda_compatability()

    # I instantiate the tokenizer as a global variable, as the .map function in transform_datasets was not working properly. 
    # This should not be an issue, as the tokenizer remains consistent during training and evaluation.
    global tokenizer
    
    ## retrieve model and tokenizer from huggingface to prepare dataset
    model, tokenizer = preprocess_model(model_name)
    
    ### retrieve the unprocessed data from the csv files
    entire_dataset = load_CACAPO_data()
    
    # ## process the dataset and split it into its natural train, val, test split
    train_ds, val_ds, test_ds, true_articles_test, test_rdf_inputs = transform_datasets(entire_dataset)

    ### setup the training arguments 
    training_args = set_training_args(model_name=model_name, learning_rate = 0.01, 
                                      num_train_epochs= 25, evaluation_strategy = 'steps', generation_num_beams=5, generation_max_length = 100, 
                                      gradient_accumulation_steps = 3, per_device_train_batch_size= 64, per_device_eval_batch_size= 64)

    ###create a trainer instance 
    trainer = set_trainer(model_name, training_args, train_ds, val_ds, tokenizer)

    # Both mt5 and T5-dutch have / in their name, which makes pathing more chaotic
    if '/' in model_name:
       path_model_name = model_name.replace('/', '_')
    elif '-' in model_name:
       path_model_name = model_name.replace('-', '_')

    ## Finally fine-tune the model and save it
    train_and_save(trainer, path_model_name)

    testset_evaluation_results = evaluate_test_set( path_model_name, test_ds, true_articles_test, test_rdf_inputs)

    return testset_evaluation_results

def main():
    global model_name
    models = ["t5-base", "yhavinga/t5-v1.1-base-dutch-cased", 'google/mt5-base']

    model_name = 'google/mt5-base' 
    results = fine_tune_model(model_name)

############################################################################## End Full fine-tune setup######################################################################################


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir '/content/drive/MyDrive/MscThesis/Tensorboard'/runs

In [ ]:
main()

Torch version: 1.12.1+cu113
Cuda version: 11.3
Cudnn version: 8302
Is cuda available: True
Number of cuda devices: 1
Current default device: 0
First cuda device: <torch.cuda.device object at 0x7f9c26b5fa50>
Name of the first cuda device: A100-SXM4-40GB




/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


LOGGING: preprocess_model DONE 

LOGGING: load_CACAPO_data DONE 



  0%|          | 0/3 [00:00<?, ?it/s]

LOGGING: transform_datasets DONE 

[2022-10-27 15:59:42,255] [INFO] [comm.py:635:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
LOGGING: set_training_args DONE 



...

LOGGING: set_trainer DONE 

[2022-10-27 15:59:49,719] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.7.4, git-hash=unknown, git-branch=unknown
[2022-10-27 15:59:54,249] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Installed CUDA version 11.2 does not match the version torch was compiled with 11.3 but since the APIs are compatible, accepting this combination
Using /root/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py37_cu113/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...
Time to load cpu_adam op: 3.3992526531219482 seconds
[2022-10-27 16:00:01,956] [INFO] [logging.py:68:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adamw as basic optimizer
[2022-1

***** Running training *****
  Num examples = 15290
  Num Epochs = 25
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 3
  Total optimization steps = 1975
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Using /root/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...
Time to load utils op: 0.0035567283630371094 seconds


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


RuntimeError: ignored

In [ ]:
def add_parent_results_to_json(path_model_name):
  with open(f'/content/drive/MyDrive/MscThesis/Logging_Results/google_mt5-base_logResults.json', 'r') as result_file:     
    overal_results = json.load(result_file)   
    #print(f'overal results:   {overal_results} \n\n\n')


  with open(f'/content/drive/MyDrive/MscThesis/Logging_Results/logResultsColabParent.json', 'r') as add_to_file:
    parent_results = json.load(add_to_file)
    #print(f'parent_results :   {parent_results} \n\n\n')

  overal_results.update(parent_results)
  #print(f'overal_results v2 :   {overal_results} \n\n\n')

  with open(f'/content/drive/MyDrive/MscThesis/Logging_Results/google_mt5-base_logResults.json', 'w') as f: 
    json.dump(overal_results, f)



In [ ]:
add_parent_results_to_json('google_mt5-base')

overal results:   {'blue_output': {'bleu': 0.0, 'precisions': [0.027972027972027972, 0.0, 0.0, 0.0], 'brevity_penalty': 0.7249313475262888, 'length_ratio': 0.7566137566137566, 'translation_length': 143, 'reference_length': 189}, 'rouge_output': {'rouge1': 0.05958574717765894, 'rouge2': 0.017592592592592594, 'rougeL': 0.04889025054466231, 'rougeLsum': 0.050389619231531}, 'meteor_results': {'meteor': 0.02181515786989303}, 'perp_output': {'perplexities': [770.9533081054688, 104.64604949951172, 587.6917724609375, 1762.919677734375, 171.27622985839844, 566.5110473632812, 171.27622985839844, 11.1726655960083, 246.65447998046875, 994.7477416992188, 513.990478515625, 336.4140625], 'mean_perplexity': 519.854478597641}, 'bertscore_output': {'precision': [0.7894889712333679, 0.7600829005241394, 0.7710351943969727, 0.7584834098815918, 0.7792298197746277, 0.7697867155075073, 0.7790033221244812, 0.7148011922836304, 0.8261810541152954, 0.7529188394546509, 0.7539244890213013, 0.7357916831970215], 'rec

## Prefix discussion

https://github.com/huggingface/transformers/issues/13249 states prefixes for mT5 are unnecessary, but not harmful. Maybe necessary for T5 and T5-dutch?

In [ ]:
{
    "fp16": {
        "enabled": "auto",
    },

    "amp": {
      "enabled": "auto",
      "opt_level": "auto"
    },
    
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },

    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu"
        },
        "offload_param": {
            "device": "cpu"
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e6,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e6,
        "stage3_max_reuse_distance": 1e6,
        "stage3_gather_16bit_weights_on_model_save": true
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}